In [1]:
from pip_flow.pip_flow import PipFlow


pip_flow=PipFlow(device='cloud', model_key="PipableAI/pipable-worflow-automaton-private-v2")

In [2]:
prompt="""
<question>
write a python function to double the input number.
</question>
<code>
"""
pip_flow.generate(prompt, eos_token="code")

'\n\n```python\ndef double_number(num):\n  return num * 2\n```\n\n'